In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101
from random_search import run_random_search, random_spec
from neural_predictor import classifier, regressor, regressor_mlp, regressor_cnn
from input_preprocessing import preprocess_nasbench

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 48 seconds


### Ablation Study 1: N vs K and Ablation Study

In [4]:
LOOPS = 60
MAX_SAMPLES = 2000
MAX_TIME_BUDGET = 8e5

In [5]:
def get_N_samples(N):
    models = []

    for _ in range(N):
        while True:
            model = random_spec(nasbench)
            if model not in models:
                models.append(nasbench.query(model))
                break
                
    return preprocess_nasbench(models)

In [6]:
results = {}

In [7]:
for N in [43, 86, 129, 172, 344, 860]:
    train_data = get_N_samples(N)
    reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_acc'])
    
    np_val_avg, np_test_avg = [], []
    np_val_std, np_test_std = [], []

    val_acc, test_acc = [], []
    for k in tqdm(range(MAX_SAMPLES - N)):
        K = k + 1

        test_models = get_N_samples(N+K)
        pred_acc = reg.predict([test_models['X'], test_models['norm_A'], test_models['norm_AT']]).ravel()

        topk_idx = tf.math.top_k(pred_acc, k=K).indices.numpy()
        selected_val = test_models['val_acc'][topk_idx]
        selected_test = test_models['test_acc'][topk_idx]

        best_val_idx = np.argmax(selected_val)

        val_acc.append(selected_val[best_val_idx])
        test_acc.append(selected_test[best_val_idx])

        np_val_avg.append(np.max(val_acc))
        np_val_std.append(np.std(val_acc))

        np_test_avg.append(np.max(test_acc))
        np_test_std.append(np.std(test_acc))
        
    results[N] = {
        'np_val_avg': np_val_avg,
        'np_test_avg': np_test_avg,
        'np_val_std': np_val_std,
        'np_test_std': np_test_std
    }

  0%|          | 0/1957 [00:00<?, ?it/s]

  0%|          | 0/1914 [00:00<?, ?it/s]

  0%|          | 0/1871 [00:00<?, ?it/s]

  0%|          | 0/1828 [00:00<?, ?it/s]

  0%|          | 0/1656 [00:00<?, ?it/s]

  0%|          | 0/1140 [00:00<?, ?it/s]

In [13]:
np.save('outputs/ablation_study_by_samples.npy', results)

### Ablation Study 2: MLP vs CNN vs GCN

In [17]:
# MLP
N = 172
train_data = get_N_samples(N)
reg = regressor_mlp(np.concatenate([train_data['X'], train_data['norm_A']], axis=-1), train_data['val_acc'])

np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for k in tqdm(range(MAX_SAMPLES - N)):
    K = k + 1

    test_models = get_N_samples(N+K)
    pred_acc = reg.predict(np.concatenate([test_models['X'], test_models['norm_A']], axis=-1)).ravel()

    topk_idx = tf.math.top_k(pred_acc, k=K).indices.numpy()
    selected_val = test_models['val_acc'][topk_idx]
    selected_test = test_models['test_acc'][topk_idx]

    best_val_idx = np.argmax(selected_val)

    val_acc.append(selected_val[best_val_idx])
    test_acc.append(selected_test[best_val_idx])

    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(val_acc))

    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(test_acc))

  0%|          | 0/1828 [00:00<?, ?it/s]

In [18]:
result_mlp = {
    'np_val_avg': np_val_avg,
    'np_test_avg': np_test_avg,
    'np_val_std': np_val_std,
    'np_test_std': np_test_std
}

In [19]:
np.save('outputs/ablation_study_by_arch_mlp.npy', result_mlp)

In [6]:
# CNN
N = 172
train_data = get_N_samples(N)
reg = regressor_cnn(np.concatenate([train_data['X'], train_data['norm_A']], axis=-1), train_data['val_acc'])

np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for k in tqdm(range(MAX_SAMPLES - N)):
    K = k + 1

    test_models = get_N_samples(N+K)
    pred_acc = reg.predict(np.concatenate([test_models['X'], test_models['norm_A']], axis=-1)).ravel()

    topk_idx = tf.math.top_k(pred_acc, k=K).indices.numpy()
    selected_val = test_models['val_acc'][topk_idx]
    selected_test = test_models['test_acc'][topk_idx]

    best_val_idx = np.argmax(selected_val)

    val_acc.append(selected_val[best_val_idx])
    test_acc.append(selected_test[best_val_idx])

    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(val_acc))

    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(test_acc))

  0%|          | 0/1828 [00:00<?, ?it/s]

In [7]:
result_cnn = {
    'np_val_avg': np_val_avg,
    'np_test_avg': np_test_avg,
    'np_val_std': np_val_std,
    'np_test_std': np_test_std
}

In [8]:
np.save('outputs/ablation_study_by_arch_cnn.npy', result_cnn)

In [9]:
# GCN
N = 172
train_data = get_N_samples(N)
reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_acc'])

np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for k in tqdm(range(MAX_SAMPLES - N)):
    K = k + 1

    test_models = get_N_samples(N+K)
    pred_acc = reg.predict([test_models['X'], test_models['norm_A'], test_models['norm_AT']]).ravel()

    topk_idx = tf.math.top_k(pred_acc, k=K).indices.numpy()
    selected_val = test_models['val_acc'][topk_idx]
    selected_test = test_models['test_acc'][topk_idx]

    best_val_idx = np.argmax(selected_val)

    val_acc.append(selected_val[best_val_idx])
    test_acc.append(selected_test[best_val_idx])

    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(val_acc))

    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(test_acc))

  0%|          | 0/1828 [00:00<?, ?it/s]

In [10]:
result_gcn = {
    'np_val_avg': np_val_avg,
    'np_test_avg': np_test_avg,
    'np_val_std': np_val_std,
    'np_test_std': np_test_std
}

In [11]:
np.save('outputs/ablation_study_by_arch_gcn.npy', result_gcn)